In [1]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


# Creating a model
A second-generation model of the main "originality score" algorithm: preprocessing a sample paper, performing analytics, saving the document's hash, and returning a score.  Uses embedding to improve understanding.

In [2]:
import warnings
warnings.filterwarnings('ignore')

import os
import nltk
import numpy as np
import pandas as pd

## Read in dataset
Load the [Reuter 50_50 training dataset](https://archive.ics.uci.edu/ml/datasets/Reuter_50_50).

TODO:  download and extract directly from website

In [3]:
DATASET_PATH = 'data/C50/C50all/'

In [4]:
# source modified from:
# https://github.com/devanshdalal/Author-Identification-task/blob/master/learner.py
authors = os.listdir(DATASET_PATH)
data = []

for author in authors:
  texts = os.listdir(DATASET_PATH + author + '/')
  for text in texts:
    f=open(DATASET_PATH + author + '/' + text, 'r')
    data.append([author, f.read()])
    f.close()
    
df = pd.DataFrame(data, columns=["author", "text"])
df.head()

,author,text
0,KeithWeir,Shares in British media group EMAP Plc tumbled...
1,KeithWeir,British media company EMAP said on Monday it e...
2,KeithWeir,British television and newspaper group United ...
3,KeithWeir,Newcastle United season ticket holders were re...
4,KeithWeir,"EMI, one of the world's top five music compani..."


## Preprocess data

In [5]:
# download 'punkt' if this is first time in notebook
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/ubuntu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [25]:
y = df["author"]

# change text to lower case, replace new lines, and tokenize
X = df["text"].str.lower().replace('\n', ' ')
X = [nltk.word_tokenize(x) for x in X]
print("First 100 tokens in text:\n", X[0][:100])

First 100 tokens in text:
 ['shares', 'in', 'british', 'media', 'group', 'emap', 'plc', 'tumbled', 'about', 'three', 'percent', 'on', 'monday', 'after', 'the', 'company', 'said', 'it', 'was', 'losing', 'managing', 'director', 'david', 'arculus', 'to', 'television', 'and', 'newspaper', 'firm', 'united', 'news', '&', 'amp', ';', 'media', '.', 'arculus', ',', '50', ',', 'who', 'joined', 'emap', 'in', '1972', ',', 'will', 'move', 'to', 'united', 'news', 'on', 'april', '7', 'as', 'chief', 'operating', 'officer', '.', 'his', 'responsibilites', 'there', 'will', 'range', 'from', 'consumer', 'publishing', 'through', 'television', 'to', 'trade', 'magazines', 'and', 'exhibitions', '.', '``', 'david', 'arculus', 'has', 'demonstrated', 'a', 'sure', 'touch', 'as', 'a', 'builder', 'of', 'media', 'businesses', '.', 'his', 'experience', 'and', 'proven', 'record', 'make', 'him', 'a', 'valuable', 'addition']


## Create vocabulary

### Reuters dataset vocabulary

In [7]:
all_text = [y for x in X for y in x]
print("Reuters dataset")
print("Total words: {}".format(len(all_text)))
print("Unique words: {}".format(len(set(all_text))))

Reuters dataset
Total words: 2902829
Unique words: 53207


In [8]:
# Create a vocab of the Reuters dataset, ordered by frequency (decending)
f = nltk.FreqDist(all_text)
reuters_most_common = [w for (w,_) in f.most_common()]
print("Top 100 Reuters words:\n", reuters_most_common[:100])

Top 100 Reuters words:
 ['the', ',', '.', 'to', 'of', 'a', 'in', 'and', 'said', "'s", "''", '``', 'on', 'for', 'that', 'is', 'it', 'with', '$', 'be', 'at', 'by', 'its', 'was', 'as', 'from', 'he', 'will', 'but', 'has', 'have', 'percent', 'would', 'are', 'million', 'not', 'which', 'an', 'year', '(', ')', 'we', 'this', 'company', 'had', 'they', 'new', 'were', 'market', 'china', 'billion', 'up', 'more', 'been', '--', 'one', 'also', 'or', 'about', 'after', 'last', 'analysts', 'than', 'their', 'over', 'some', 'u.s.', 'hong', 'there', 'kong', 'could', 'who', 'two', 'i', 'group', 'business', 'share', 'first', 'other', 'his', 'government', 'companies', 'industry', 'bank', 'if', 'stock', 'expected', 'into', 'out', 'years', 'sales', 'shares', 'analyst', 'told', 'chinese', 'no', 'when', 'all', 'people', ';']


### Google's top 10,000 word vocabulary

In [9]:
# Load top 10,000 English words, according to Google
#    Source:  https://github.com/first20hours/google-10000-english
with open('data/google-10000-english-usa-no-swears.txt', 'r') as f:
  google_most_common = f.read().replace('\n', ' ')
google_most_common = nltk.word_tokenize(google_most_common)
print("Top 100 Google words:\n", google_most_common[:100])

Top 100 Google words:
 ['the', 'of', 'and', 'to', 'a', 'in', 'for', 'is', 'on', 'that', 'by', 'this', 'with', 'i', 'you', 'it', 'not', 'or', 'be', 'are', 'from', 'at', 'as', 'your', 'all', 'have', 'new', 'more', 'an', 'was', 'we', 'will', 'home', 'can', 'us', 'about', 'if', 'page', 'my', 'has', 'search', 'free', 'but', 'our', 'one', 'other', 'do', 'no', 'information', 'time', 'they', 'site', 'he', 'up', 'may', 'what', 'which', 'their', 'news', 'out', 'use', 'any', 'there', 'see', 'only', 'so', 'his', 'when', 'contact', 'here', 'business', 'who', 'web', 'also', 'now', 'help', 'get', 'pm', 'view', 'online', 'c', 'e', 'first', 'am', 'been', 'would', 'how', 'were', 'me', 's', 'services', 'some', 'these', 'click', 'its', 'like', 'service', 'x', 'than', 'find']


### Compare vocabs and choose one for this project

In [10]:
# Look through the differences between the two vocabs
unique_reuters_words = [x for x in reuters_most_common[:10000] if x not in google_most_common]
print("Reuters unique words: {}. Here's the top 100.".format(len(unique_reuters_words)))
print(unique_reuters_words[:100])

unique_google_words = [x for x in google_most_common if x not in reuters_most_common[:10000]]
print("\nGoogle unique words: {}. Here's the top 100.".format(len(unique_google_words)))
print(unique_google_words[:100])

Reuters unique words: 4390. Here's the top 100.
[',', '.', "'s", "''", '``', '$', '(', ')', '--', 'u.s.', ';', '&', "n't", '1996', "'", 'corp.', '1997', 'inc.', '...', '10', 'tonnes', '1995', '20', "'re", 'pence', '30', 'bre-x', '1', 'deng', '50', 'mci', ':', 'newsroom', '15', 'tung', '100', 'boeing', 'wang', 'co.', '25', 'takeover', '12', 'francs', 'traders', 'yuan', "'ve", '40', 'labour', 'rival', 'conrail', 'margins', 'busang', 'airbus', 'shareholder', 'ltd.', 'nomura', '1995.', '14', 'cocoa', 'stg', 'jiang', '-', '11', 'handover', '1994', 'long-term', '1996/97', '300', '2', '60', 'klaus', 'speculation', 'uaw', 'crowns', 'privatisation', '1998', '1997.', 'regulators', '90', 'barrick', 'long-distance', 'thomson-csf', 'brokerage', '1996.', 'spokeswoman', '18', '200', '?', 'tonne', 'jumped', '171', 'csx', '31', 'rivals', '16', "'ll", '13', 'telecoms', '=', 'natwest']

Google unique words: 4279. Here's the top 100.
['pm', 'click', 'x', 're', 'info', 'ebay', 'dvd', 'website', 'v', 'descr

Let's take a look at some of these unique words, in order of frequency, to see if they are domain-specific.  For the Reuters vocab, some are punctuation, many are numbers, and the remainder are mostly domain-specific (international-business) related words, such as *privitisation, pre-tax*, and *conglomerate* or names such as *murdoch* and *monsanto*.  For the Google vocab, some are letters, some computer-related such as *forums* and *login*, and some are more general such as *color* and *thank*.

Let's use the Google vocab and add punctuations and contractions.

In [11]:
# Extend common vocab to include punctuation + contractions
from string import punctuation
vocab = google_most_common + list(punctuation)

## Convert text and authors to network-ready input
### Download embedding model to represent words

In [12]:
import gensim.downloader as api
# https:/github.com/RaRe-Technologies/gensim-data
# glove-twitter-25
# word2vec-google-news-300
info = api.info()
embed_model = api.load("glove-twitter-100")

In [36]:
embed_size = embed_model.vector_size

In [13]:
# print sample data
print("Embedding for 'house':\n", embed_model.wv['house'])
print("\nSimilar words to 'house':\n", embed_model.most_similar("house"))

Embedding for 'house':
 [-1.3345e-01  3.4688e-01  3.0748e-01 -2.1794e-03  7.1898e-01 -2.8725e-03
  9.5989e-02  5.5276e-01  1.2153e-01 -2.6555e-01 -1.0277e+00  7.2278e-01
 -4.2767e+00 -9.0406e-02  1.1909e-01 -5.0647e-02 -3.3165e-01 -1.8213e-01
 -3.6218e-01  6.9813e-03  2.0147e-01 -2.9150e-01 -1.6417e-01 -2.8022e-01
  5.4800e-01 -5.8081e-01  3.8146e-01 -5.5519e-01  1.6094e-01 -5.2039e-02
 -1.4798e-01  1.0892e-03 -2.6702e-01 -1.7885e-01  5.1449e-02  6.7434e-02
  9.5654e-02  5.6137e-01  7.1208e-03  4.7000e-01 -3.1460e-01  1.0552e+00
  5.2215e-01 -4.8432e-01  2.8615e-01  7.9474e-02  6.4211e-01  6.5274e-01
 -2.6493e-01 -8.9566e-02 -2.6298e-01 -3.4906e-01  3.3645e-02  2.1278e-01
 -1.0738e+00 -3.6867e-01  1.8473e-01  3.3821e-01  5.7516e-01  1.7559e-01
 -1.5436e-01  5.2836e-02 -9.8523e-02 -4.0975e-01 -8.5839e-02 -3.1527e-01
  1.7936e-01 -2.0953e-01  6.6424e-01 -5.7412e-02  2.4528e-01 -2.2577e-01
 -3.3233e-01  2.1225e-01  2.3743e-01  1.3298e-01 -4.4889e-01  4.9577e-01
  4.3360e-01  2.4248e-01  1

### Convert text to embedding vectors

In [27]:
from tqdm import tqdm, tqdm_notebook

# filter rare words and convert to vectors
for i, doc in enumerate(tqdm(X)):
    X[i] = [embed_model[word] for word in doc if word in vocab and word in embed_model]
        
print("Embedded vector representation of some words of the first text:\n", X[0][0:3])

100%|██████████| 5018/5018 [02:25<00:00, 34.44it/s]

Embedded vector representation of some words of the first text:
 [array([ 0.36901  , -0.097805 ,  0.77436  , -0.66244  , -0.1459   ,
       -0.92174  , -0.059589 , -0.74321  ,  0.1989   , -0.69029  ,
       -0.29205  ,  0.017334 , -2.4598   ,  0.17755  ,  0.038644 ,
        0.066692 ,  0.043657 , -0.094778 ,  0.074137 , -0.43868  ,
        0.35035  , -0.59119  ,  0.06502  , -0.13868  , -0.87692  ,
        0.83511  , -0.024205 ,  0.17247  ,  0.33899  , -0.074402 ,
        0.54115  ,  0.25162  ,  0.10621  , -0.27057  ,  1.0452   ,
        0.010004 , -0.43414  ,  0.1705   , -0.37303  , -1.3054   ,
        0.68982  , -0.27434  ,  0.36169  ,  0.12466  ,  0.5389   ,
        0.11821  ,  0.40161  , -0.1011   , -0.46405  ,  0.65829  ,
       -0.041556 , -0.68439  , -0.57501  ,  0.89671  , -0.19499  ,
        0.4853   ,  0.41231  ,  0.064372 ,  0.29773  , -0.61602  ,
       -0.80376  , -0.76901  ,  0.079914 , -0.014108 ,  0.89144  ,
        0.27667  ,  0.21634  ,  0.46204  , -0.26506  , -0.10666

### Windowing
Creating smaller windows of data to process

In [49]:
# Create 100 word chucks
WINDOW_SIZE = 200
WINDOW_SPACING = 50

def chunk(x, y):
    X_chunk = []
    y_chunk = []
    
    for i in range(0, len(x)-WINDOW_SIZE, WINDOW_SPACING):
        X_chunk.append(x[i:i+WINDOW_SIZE])
        y_chunk.append(y)
    
    return X_chunk, y_chunk

X_chunks = []
y_chunks = []
for i, _ in enumerate(X):
    xc, yc = chunk(X[i], y[i])
    X_chunks += xc
    y_chunks += yc

In [50]:
X_chunks = np.array(X_chunks)
y_chunks = np.array(y_chunks)
print("X_chunks shape:", X_chunks.shape)
print("y_chunks shape:", y_chunks.shape)

X_chunks shape: (31591, 200, 100)
y_chunks shape: (31591,)


## Training

### Create training, test, and "new" sets

In [51]:
from sklearn.model_selection import train_test_split

# Keeps some authors aside for hash testing
x_train, x_new, y_train, y_new = train_test_split(X_chunks, y_chunks, train_size=0.8, shuffle=False)

# Split remainder into 70% training and 30% testing and shuffle
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, train_size=0.9, random_state=1)

print("New:   {} text from {} authors".format(x_new.shape[0], len(np.unique(y_new, axis=0))))
print("Train: {} text from {} authors".format(x_train.shape[0], len(np.unique(y_train, axis=0))))
print("Test:  {} text from {} authors".format(x_test.shape[0], len(np.unique(y_test, axis=0))))

New:   6319 text from 10 authors
Train: 22744 text from 41 authors
Test:  2528 text from 41 authors


### One-hot encode labels (authors)

In [52]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

#encoder = LabelEncoder()
#encoded = encoder.fit_transform(df["author"])
#y = to_categorical(encoded)

encoder = LabelEncoder()
encoded = encoder.fit_transform(y_train)
y_train = to_categorical(encoded)
y_test = to_categorical(encoder.transform(y_test))

print("Author {} is one-hot encoded as:\n".format(df["author"][0]), y_train[0])

Author KeithWeir is one-hot encoded as:
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [34]:
output_shape = len(y_train[0])
print("Network output shape (# of trained authors):", output_shape)

Network output shape (# of trained authors): 41


### Free memory

In [ ]:
del X 
del y 
del embed_model 
del df
del X_chunks
del y_chunks
del data

### Create network model

In [53]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, Bidirectional
    
model = Sequential()
#model.add(LSTM(256, dropout=0.3, input_shape=(WINDOW_SIZE, embed_size), return_sequences=False))
model.add(LSTM(256, dropout=0.5, recurrent_dropout=0.2, input_shape=(WINDOW_SIZE, embed_size), return_sequences=True))
model.add(LSTM(256, dropout=0.2))
model.add(Dense(output_shape, activation='softmax', name='output'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
#model.summary()

In [ ]:
model.fit(x_train, 
          y_train,
          batch_size=512,
          epochs=50,
          validation_split=0.2,
          shuffle=True)

Train on 18195 samples, validate on 4549 samples
Epoch 1/50
18195/18195 [==============================] - 53s 3ms/step - loss: 3.6242 - acc: 0.0499 - val_loss: 3.3537 - val_acc: 0.0842
Epoch 2/50
18195/18195 [==============================] - 49s 3ms/step - loss: 3.3630 - acc: 0.0901 - val_loss: 3.0812 - val_acc: 0.1438
Epoch 3/50
18195/18195 [==============================] - 49s 3ms/step - loss: 3.2269 - acc: 0.1084 - val_loss: 3.0587 - val_acc: 0.1416
Epoch 4/50
18195/18195 [==============================] - 49s 3ms/step - loss: 3.1675 - acc: 0.1196 - val_loss: 2.9433 - val_acc: 0.1570
Epoch 5/50
18195/18195 [==============================] - 49s 3ms/step - loss: 3.3469 - acc: 0.1040 - val_loss: 3.2160 - val_acc: 0.1139
Epoch 6/50
18195/18195 [==============================] - 49s 3ms/step - loss: 3.2064 - acc: 0.1250 - val_loss: 2.9284 - val_acc: 0.1759
Epoch 7/50
18195/18195 [==============================] - 49s 3ms/step - loss: 3.0397 - acc: 0.1514 - val_loss: 2.7990 - val_acc:

In [47]:
score, acc = model.evaluate(x_test, y_test)
print('Test score:', score)
print('Test accuracy:', acc)

3323/3323 [==============================] - 18s 6ms/step
Test score: 0.8724555654443964
Test accuracy: 0.7123081552634353


In [48]:
# save model
model.save("data/5-100-lstm256-256-model.h5")